In [6]:
import pandas as pd
import numpy as np
import pickle
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [62]:
address_data = pd.read_csv('../data/sc122a.txt', delimiter = '\t', header = 0, encoding='latin-1')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5,6,9,22,23,26,28,54,55,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [63]:
street = address_data['LSTREE']
city = address_data['LCITY']
zipcode = list(map(str, address_data['LZIP'])) #convert all int in LZIP to strings so they can be appended 
state = address_data['LSTATE']

In [64]:
def make_address(street, city, state, zipcode): #returns a new list of complete addresses with combined street, city, state, and zipcode 
    addresses = [] #empty list to fill with full addresses
    for i in np.arange(len(street)): 
        full_address = street[i].lower() + ' ' + city[i].lower() + ' ' + state[i].lower() + ' ' + zipcode[i].lower() #for every street in list of streets, append city, state, and zip to create full address
        addresses.append(full_address) #append full address into empty list of addresses
    return addresses 

In [76]:
full_address = make_address(street, city, state, zipcode)
full_address

['rt 2 old springville rd pinson al 36126',
 '315 outback rd clayton al 36016',
 'industrial school rd mt meigs al 36057',
 'industrial school rd mt meigs al 36057',
 'i85 service rd mount meigs al 36057',
 '8950 roebuck blvd birmingham al 35206',
 '1299 hillsboro parkway helena al 35080',
 '600 east alabama ave albertville al 35950',
 '402 east mccord ave albertville al 35950',
 '901 west mckinney ave albertville al 35950',
 '1100 horton rd albertville al 35950',
 '257 country club rd albertville al 35951',
 '1100 horton rd albertville al 35950',
 '6077 main st grant al 35747',
 '1990 asbury rd albertville al 35950',
 '140 claysville school rd guntersville al 35976',
 'hwy 75 douglas al 35964',
 'hwy 75 douglas al 35964',
 '2233 shoal creek rd arab al 35016',
 '6077 main st grant al 35747',
 '3685 union grove rd union grove al 35175',
 '12312 us hwy 431 south guntersville al 35976',
 '12312 us hwy 431 south guntersville al 35976',
 '200 bethlehem rd horton al 35950',
 '1050 scant city

In [66]:
address_data['ADDRESS13'] = full_address

In [77]:
address_NCESSCH_data = address_data[['NCESSCH', 'ADDRESS13']] #select only NCESSCH and ADRESS13 columns because we're joining on NCESSCH and we only want to add the ADDRESS13 values to charters_2015.pkl 
address_NCESSCH_data


,NCESSCH,ADDRESS13
0,10000200277,rt 2 old springville rd pinson al 36126
1,10000201402,315 outback rd clayton al 36016
2,10000201667,industrial school rd mt meigs al 36057
3,10000201670,industrial school rd mt meigs al 36057
4,10000201705,i85 service rd mount meigs al 36057
5,10000201706,8950 roebuck blvd birmingham al 35206
6,10000201876,1299 hillsboro parkway helena al 35080
7,10000500870,600 east alabama ave albertville al 35950
8,10000500871,402 east mccord ave albertville al 35950
9,10000500879,901 west mckinney ave albertville al 35950


In [14]:
CER = pd.read_csv('CER_2012-13.csv', encoding = 'latin-1') #open CER file 

In [88]:
for i in np.arange(len(CER['CER_ADDRESS'])):
    CER.loc[i,'CER_ADDRESS'] = CER.loc[i,'CER_ADDRESS'].replace(',', '').replace('.','').lower() #loop through every address, removing commas and periods, and converting all letters to lowercase


In [16]:
charters_nonduplicate = pd.read_csv('../../nowdata/parsing/overlaps_removed_df.csv', sep = '\t', header = 0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (128,129,130,131,132,133,134,135,137,196,197,198,211,299,300,321,335) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [78]:
charters_nonduplicate_with_address = pd.merge(charters_nonduplicate, address_NCESSCH_data, how = 'left', on = 'NCESSCH') #merge addresses to charters_nonduplicate based on NCESSCH 

In [87]:
charters_nonduplicate_with_address['SCHNAM12'] = [str(name).lower() for name in charters_nonduplicate_with_address['SCHNAM12']]

In [19]:
CER['CER_NAME'] = [str(name).lower() for name in CER['CER_NAME']]

In [79]:
charters_nonduplicate_with_address['ADDRESS13']

0                                                      NaN
1                          1010 fourth ave bethel ak 99559
2                         410 schoenbar ketchikan ak 99901
3                      410 schoenbar rd ketchikan ak 99901
4                    1705 west 32nd ave anchorage ak 99517
5                  401 east fireweed ln anchorage ak 99503
6                                                      NaN
7                                                      NaN
8                  4802 bryn mawr court anchorage ak 99508
9                     10901 mausel st eagle river ak 99577
10        400 west northern lights blvd anchorage ak 99503
11            5530 east northern lights anchorage ak 99504
12       650 west international airport anchorage ak 99507
13                       110 muldoon rd anchorage ak 99504
14                                                     NaN
15                           430 fourth st juneau ak 99801
16                          705 frontage rd kenai ak 996

In [108]:
charters_merge_CER = pd.merge(charters_nonduplicate_with_address, CER, how = 'left', left_on = ['ADDRESS13','SCHNAM12'], right_on = ['CER_ADDRESS','CER_NAME']) #merge based on columns ADDRESS13, SCH_NAME, and LEVEL in charters_nonduplicate_with_address to columns CER_ADDRESS, CER_NAME, and CER_GRADE_CATEGORIES in CER 

In [111]:
charters_merge_CER_schname = pd.merge(charters_nonduplicate_with_address, CER, how = 'left', left_on = ['SCHNAM12'], right_on = ['CER_NAME'])
charters_merge_CER_schname.drop_duplicates(subset = 'NCESSCH', keep = 'first', inplace = True)
print(len(charters_merge_CER_schname['NCESSCH']))
print(sum(charters_merge_CER_schname['NCESSCH'].duplicated()))


10965
0


In [46]:
charters_merge_CER_schname

,"% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)","% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)",% Civilian Population in Labor Force 16 Years and Over: Employed,% Civilian Population in Labor Force 16 Years and Over: Unemployed,"% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining","% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and Accommodation and Food Services",% Employed Civilian Population 16 Years and Over: Construction,"% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social Assistance","% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental and Leasing",% Employed Civilian Population 16 Years and Over: Information,...,prog_strength_y,ADDRESS13,CER_NAME,CER_ADDRESS,CER_URL,CER_DESCRIPTION,CER_OPENED,CER_MEMBER,CER_GRADES,CER_GRADE_CATEGORIES
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,88.03,11.97,81.70,18.30,0.83,3.50,3.89,36.46,3.12,1.71,...,0.034483,1010 fourth ave bethel ak 99559,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.105839,410 schoenbar ketchikan ak 99901,ketchikan charter school,410 schoenbar road ketchikan ak 99901,http://kcs.kgbsd.org/#,"Fosters growth of character, high academic ach...",1998,179.0,K-8,"Elementary-Middle,"
3,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.087500,410 schoenbar rd ketchikan ak 99901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.087179,1705 west 32nd ave anchorage ak 99517,aquarian charter school,1705 w 32nd ave anchorage ak 99517,http://www.aquariancharterschool.com/#,"Includes drama, music, Spanish Inclusion and a...",1997,360.0,K-6,"Elementary-Middle,"
5,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.028571,401 east fireweed ln anchorage ak 99503,family partnership charter school,401 e fireweed ln anchorage ak 99503,http://www.fpcs.net/#,Partnership school wherein parents bear the pr...,1997,591.0,K-12,"Elementary-Middle, Middle-High,"
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.320000,4802 bryn mawr court anchorage ak 99508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.101796,10901 mausel st eagle river ak 99577,eagle academy charter school,10901 mausel st ste 101 eagle river ak 99577,http://www.eagleacademycharterschool.com/#,Uses the Spalding method of teaching across cu...,2005,163.0,K-6,"Elementary-Middle,"


In [112]:
non_null_or_duplicate_vals = set(charters_merge_CER_schname['CER_URL'].dropna())
len(non_null_or_duplicate_vals)

1844

In [40]:
CER_columns = list(CER.columns) #list of all columns labels in CER
CER_columns.remove('CER_URL') #remove CER_URL from the list 

In [41]:
charters_merge_CER = charters_merge_CER.drop(labels = CER_columns, axis = 1) #drop all columns in CER except CER_URL 

In [42]:
len(charters_nonduplicate_with_address) == len(charters_merge_CER) #merged dataframe has same amount of rows as charters_nonduplicate_with_address

False

In [43]:
charters_merge_CER

,"% Civilian Population 16 to 19 Years: High School Graduate, or Enrolled (in School)","% Civilian Population 16 to 19 Years: Not High School Graduate, Not Enrolled (Dropped Out)",% Civilian Population in Labor Force 16 Years and Over: Employed,% Civilian Population in Labor Force 16 Years and Over: Unemployed,"% Employed Civilian Population 16 Years and Over: Agriculture, Forestry, Fishing and Hunting, and Mining","% Employed Civilian Population 16 Years and Over: Arts, Entertainment, and Recreation, and Accommodation and Food Services",% Employed Civilian Population 16 Years and Over: Construction,"% Employed Civilian Population 16 Years and Over: Educational Services, and Health Care and Social Assistance","% Employed Civilian Population 16 Years and Over: Finance and Insurance, and Real Estate and Rental and Leasing",% Employed Civilian Population 16 Years and Over: Information,...,close_rate15,close_rate16,close_rate99,ess_strength_x,ess_strength_y,index,prog_strength_x,prog_strength_y,ADDRESS13,CER_URL
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.000000,0.000000,NaN,NaN,11229.0,NaN,NaN,nan,NaN
1,88.03,11.97,81.70,18.30,0.83,3.50,3.89,36.46,3.12,1.71,...,0.0000,0.000000,0.000000,0.137931,0.137931,6555.0,0.034483,0.034483,1010 fourth ave bethel ak 99559,http://www.yupik.org/#
2,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.0000,0.100000,0.000000,0.041058,0.041058,6557.0,0.105839,0.105839,410 schoenbar ketchikan ak 99901,NaN
3,89.50,10.50,92.43,7.57,4.12,7.45,7.89,21.39,5.10,2.36,...,0.0000,0.100000,0.000000,0.037500,0.037500,6556.0,0.087500,0.087500,410 schoenbar rd ketchikan ak 99901,NaN
4,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0000,0.000000,0.010000,0.095385,0.095385,6565.0,0.087179,0.087179,1705 west 32nd ave anchorage ak 99517,NaN
5,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0000,0.000000,0.010000,0.114286,0.114286,6564.0,0.028571,0.028571,401 east fireweed ln anchorage ak 99503,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.000000,0.010000,NaN,NaN,3754.0,NaN,NaN,nan,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,0.000000,0.010000,NaN,NaN,3755.0,NaN,NaN,nan,NaN
8,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0000,0.000000,0.010000,0.040000,0.040000,6563.0,0.320000,0.320000,4802 bryn mawr court anchorage ak 99508,NaN
9,96.88,3.12,93.95,6.05,3.63,10.46,6.39,23.01,4.86,2.44,...,0.0000,0.000000,0.010000,0.299401,0.299401,6566.0,0.101796,0.101796,10901 mausel st eagle river ak 99577,NaN


In [19]:
#charters_merge_CER.to_csv('../../nowdata/backups/charters_full_250_CER_URLs.pkl')

In [109]:
len(charters_merge_CER['NCESSCH'])

10966

In [110]:
non_null_or_duplicate_vals = set(charters_merge_CER['CER_URL'].dropna())
len(non_null_or_duplicate_vals)

440

1. List fuzzy ratio for each school name and address on charters to each school name and address on CER 
2. For each school name on charters, keep the highest fuzzy ratio 
3. If fuzzy ratio is over 70, change the school name to its match 
4. Merge based on edited data 

In [84]:
charters_nonduplicate_with_address['ADDRESS13'] = [str(address) for address in charters_nonduplicate_with_address['ADDRESS13']]

In [107]:
index = 0
for name in charters_nonduplicate_with_address.iloc[0:500]['SCHNAM12']:
    print(index)
    rationame = [fuzz.ratio(name, CERname) for CERname in CER['CER_NAME']]
    greatest_match_index_name = np.asarray(rationame).argmax()
    print(greatest_match_index_name)
    greatest_match_name = rationame[greatest_match_index_name]
    if greatest_match_name >= 70:
        print('name match found')
        if fuzz.ratio(charters_nonduplicate_with_address.loc[index, 'ADDRESS13'], CER.loc[greatest_match_index_name, 'CER_ADDRESS']) >= 70:
            print('address match found')
            charters_nonduplicate_with_address.loc[index, 'SCHNAME12'] = CER.loc[greatest_match_index_name, 'CER_NAME']
    index = index + 1
        

0
2672
1
523
2
2889
name match found
address match found
3
5599
name match found
address match found
4
368
name match found
address match found
5
1915
name match found
address match found
6
2672
7
2672
8
6073
name match found
9
1638
name match found
address match found
10
2049
name match found
address match found
11
2405
name match found
12
4769
name match found
13
214
name match found
address match found
14
2672
15
2841
name match found
address match found
16
505
name match found
address match found
17
1936
name match found
address match found
18
5148
name match found
address match found
19
2853
name match found
address match found
20
45
name match found
address match found
21
3649
name match found
address match found
22
279
name match found
23
5672
name match found
address match found
24
2672
25
2049
name match found
26
663
name match found
address match found
27
2672
28
4783
name match found
29
1129
name match found
address match found
30
2672
31
5309
name match found
address match 

2260
name match found
348
1447
name match found
349
2896
name match found
address match found
350
2672
351
3124
name match found
352
838
name match found
address match found
353
838
name match found
354
2672
355
1559
name match found
address match found
356
4897
name match found
address match found
357
2672
358
5332
name match found
address match found
359
1028
name match found
360
1689
name match found
address match found
361
3267
name match found
address match found
362
2672
363
3268
name match found
address match found
364
5409
name match found
365
2672
366
2672
367
5186
name match found
address match found
368
2672
369
4414
name match found
370
111
name match found
address match found
371
112
name match found
372
110
name match found
address match found
373
3790
name match found
address match found
374
2672
375
2201
name match found
address match found
376
2672
377
1511
name match found
address match found
378
5804
name match found
address match found
379
3785
name match found
addr

In [95]:
charters_nonduplicate_with_address.loc[1, 'ADDRESS13']

'1010 fourth ave bethel ak 99559'

In [94]:
CER.loc[523, 'CER_ADDRESS']

'1010 fourth ave bethel ak 99559'

In [103]:
fuzz.ratio(charters_nonduplicate_with_address.loc[2, 'ADDRESS13'], CER.loc[2889, 'CER_ADDRESS'])

93

In [102]:
fuzz.ratio(charters_nonduplicate_with_address.loc[2, 'SCHNAM12'], CER.loc[2889, 'CER_NAME'])

100